# How to display IFC data in Speckle

## Summary
This notebook is a tutorial about learning how to display Objects and their data in the Speckle viewer.
Speckle is an open-source platform for sharing and managing data between different software tools, 
enabling real-time collaboration and seamless data exchange in architecture, engineering, and construction projects.

This tutorial covers three excercises (Examples) for you to discover te possibility of viewing IFC Objects and their associated data in Speckle
    Example 1: One column (Send one column from the IFC file to speckle)
    Example 2: All columns (Display all columns coloured after their uniqe type in speckle)
    Example 3: slabs (Display slabs in Speckle coloured by type)

## Get started
First, set up a Speckle account by installing a Speckle manager and connecting it to your personal account. Under https://app.speckle.systems/projects create/chose a project in which you want to recieve the Data. Once you click on the Project of your choice the speckle stream ID appears in the URL https://app.speckle.systems/projects/2a3df00e3e (The number-letter combination after projects/)

Now, that you have your stream ID you are almost ready to go. You just need to create an access token. To do so, visit the website https://speckle.xyz/profile , scroll down to Developper Settings and create a new access token. Once it is created save it somwhere where you always find it again since it only shows once!. Further guidence can be found here:https://speckle.guide/dev/tokens.html.

When this is done, other people can use your access token to send data (Remember though, they send it through your account (your token is your confidential data))
The Output of the script we will build together, will be a link to the chosen stream in the speckle viewer where your objects of choice will be displayed. 

Now you are ready to start:D

First, install the necessary python packages by running these lines in your vscode terminal:

* pip install ifcopenshell

* pip install specklepy

Now you are able to import the different python libraries that you will need in this tutorial:

In [ ]:
import ifcopenshell
import ifcopenshell.geom
from specklepy.api.client import SpeckleClient
from specklepy.objects.geometry import Mesh
from specklepy.objects import Base
from specklepy.transports.server import ServerTransport
from specklepy.api.operations import send
import random

# 1. Open the IFC file
In this tutorial we are going to use the ifc model for structural group of the course 41934 Advanced BIM of 2024, so make sure that you have 
downloaded the file CES_BLD_24_06_STR.ifc and insert your file path on the following line (you could also use another .ifc file, we just can't make sure that you will not have a missing data issue):

In [109]:
ifc_file_path = '/Users/ioschkagautier/Desktop/DTU/Advanced BIM/CES_BLD_24_06_STR.ifc'
ifc_file = ifcopenshell.open(ifc_file_path)

# 2. Extract all columns + colors
Let's begin by extracting the columns from our Ifc model. We also create a main Base object to hold all columns.

In [110]:
columns = ifc_file.by_type("IfcColumn")
if len(columns) == 0:
    raise Exception("No columns found in the IFC file.")

speckle_columns = Base()

We are also going to display them in certain colors, so let's create a dictionary to hold unique colors for each combination of ObjectType (this will be useful when we will have different ObjectTypes, cf. 5. Example 2) and a function that generates a random color in hexadecimal format.

In [111]:
type_colors = {}

def generate_random_color():
    return int(f'{random.randint(0, 255):02x}{random.randint(0, 255):02x}{random.randint(0, 255):02x}', 16)

# 3. Calculate bounding box dimensions
Create a function that is going to return the dimensions of an object from its given vertices by measuring distances between them.

In [112]:
def calculate_bounding_box(vertices):
    if not vertices or not isinstance(vertices, (tuple, list)):
        return None, None, None

    # Convert vertices from tuple to a list of (x, y, z) tuples
    vertices_list = [(vertices[i], vertices[i + 1], vertices[i + 2]) for i in range(0, len(vertices), 3)]

    # Extract x, y, z values from the vertices
    x_values = [v[0] for v in vertices_list]
    y_values = [v[1] for v in vertices_list]
    z_values = [v[2] for v in vertices_list]

    min_x, max_x = min(x_values), max(x_values)
    min_y, max_y = min(y_values), max(y_values)
    min_z, max_z = min(z_values), max(z_values)

    length = max_x - min_x
    width = max_y - min_y
    height = max_z - min_z

    return length, width, height

# 4. Example 1 : one column
Let's try by displaying just the first column in Speckle. The following code extract the vertices of the column to then give it to the calculate_bounding_box function to get its geometry. We also make sure to get its ObjectType.

In [113]:
column = columns[0]
settings = ifcopenshell.geom.settings()  # Default settings for geometry extraction
shape = ifcopenshell.geom.create_shape(settings, column)  # Extract geometry

# Get the geometry vertices and faces
vertices = shape.geometry.verts
faces = shape.geometry.faces

if not vertices or not faces:
    print(f"No valid geometry found for column: {column.Name}")

# Calculate bounding box dimensions to determine dimensions
dimensions = calculate_bounding_box(vertices)
if dimensions is None:
    print(f"Could not calculate dimensions for column: {column.Name}")

# Store dimensions into variables
length, width, height = dimensions
length = round(length, 2)
width = round(width, 2)
height = round(height, 2)

# Get ObjectType (or a fallback if not available)
object_type = getattr(column, 'ObjectType', None)
if not object_type:
    object_type = column.Name if column.Name else "Unnamed"

# 4.1 Speckle mesh generation 
These following lines create a mesh to be able to display the column as a 3D object in Speckle overview.

In [ ]:
# Scale the vertices (if needed)
scaled_vertices = [v * 1 for v in vertices]  # Scale to meters for Speckle

# Format the faces for Speckle
formatted_faces = []
for i in range(0, len(faces), 3):
    formatted_faces.extend([3, faces[i], faces[i + 1], faces[i + 2]])

# Create a Mesh for this column
speckle_mesh = Mesh(
    name=f"IFC Column - {column.Name}",
    vertices=scaled_vertices,
    faces=formatted_faces
)

# 4.2 Adding color and other attributes

In [115]:
 # Apply the color to the mesh
speckle_mesh.colors = [generate_random_color()] * (len(scaled_vertices) // 3)  # Same color for all vertices

# Assign other direct attributes that are going to be displayed in Speckle
speckle_mesh['length'] = length
speckle_mesh['width'] = width
speckle_mesh['height'] = height

# Create a Speckle Base object for the column, which includes the mesh and GUID
column_object = Base()
column_object['mesh'] = speckle_mesh
column_object['guid'] = column.GlobalId if column.GlobalId else "No GUID"
column_object['name'] = column.Name if column.Name else "Unnamed Column"
column_object['ifc_type'] = column.is_a()  # e.g., IfcColumn
column_object['length'] = length  # Add length to the column object
column_object['width'] = width  # Add width to the column object
column_object['height'] = height  # Add height to the column object
column_object['object_type'] = object_type  # Add ObjectType to the column object

# Add this column object to the main collection of columns
speckle_columns[f'column_{column.GlobalId}'] = column_object

# 4.3 Python to Speckle transport
The following script is how you send the python data to the Speckle overview and how you can generate a link to easily consult it on your web browser.

In [ ]:

client = SpeckleClient(host="app.speckle.systems")  # Make sure to specify the correct server host (e.g., app.speckle.systems)

# Replace this with your actual personal access token (PAT) 
# Here you can also use default account method and run it with the already implemented token
personal_access_token = "e84c64a17e9103569a55ccb97e2f00526982240229"  

# Authenticate with the token
client.authenticate_with_token(personal_access_token)

# Set up transport to the stream
stream_id = "2a3df00e3e"  # Your specified stream ID 
transport = ServerTransport(client=client, stream_id=stream_id)

# Send the column as one object to Speckle
object_id = send(base=speckle_columns, transports=[transport])
print(f"The first column with color sent to Speckle with object ID: {object_id}")

# Print the Speckle viewer URL to view the object
print(f"View it at: https://app.speckle.systems/streams/{stream_id}/objects/{object_id}")

The first column with color sent to Speckle with object ID: 4ad88747ccf463fddf612ce747668721
View it at: https://app.speckle.systems/streams/2a3df00e3e/objects/4ad88747ccf463fddf612ce747668721


# 5. Example 2 : all columns
Now you are able to display all the columns in the same Speckle overview by following the same method just by adding a big loop over all columns and also by adaptating one new thing which is the unique key based on each ObjectType since we have different column types.

In [117]:
for column in columns:

    ### 1. EXTRACT COLUMN GEOMETRY
    settings = ifcopenshell.geom.settings()
    shape = ifcopenshell.geom.create_shape(settings, column)
    vertices = shape.geometry.verts
    faces = shape.geometry.faces

    if not vertices or not faces:
        print(f"No valid geometry found for column: {column.Name}")
        continue

    dimensions = calculate_bounding_box(vertices)
    if dimensions is None:
        print(f"Could not calculate dimensions for column: {column.Name}")
        continue

    length, width, height = dimensions
    length = round(length, 2)
    width = round(width, 2)
    height = round(height, 2)

    object_type = getattr(column, 'ObjectType', None)
    if not object_type:
        object_type = column.Name if column.Name else "Unnamed"
    

    ### 2. COLORS AND SPECKLE MESH GENERATION
    key = (object_type)

    # Assign a unique color for each key if it doesn't already exist
    if key not in type_colors:
        type_colors[key] = generate_random_color()  # Assign a random color for each unique key

    scaled_vertices = [v * 1 for v in vertices]

    formatted_faces = []
    for i in range(0, len(faces), 3):
        formatted_faces.extend([3, faces[i], faces[i + 1], faces[i + 2]])

    speckle_mesh = Mesh(
        name=f"IFC Column - {column.Name}",
        vertices=scaled_vertices,
        faces=formatted_faces
    )

    # Apply the color to the mesh (applying color based on ObjectType)
    speckle_mesh.colors = [type_colors[key]] * (len(scaled_vertices) // 3)

    speckle_mesh['length'] = length
    speckle_mesh['width'] = width
    speckle_mesh['height'] = height

    column_object = Base()
    column_object['mesh'] = speckle_mesh
    column_object['guid'] = column.GlobalId if column.GlobalId else "No GUID"
    column_object['name'] = column.Name if column.Name else "Unnamed Column"
    column_object['ifc_type'] = column.is_a() 
    column_object['width'] = width  
    column_object['height'] = height 
    column_object['object_type'] = object_type  

    speckle_columns[f'column_{column.GlobalId}'] = column_object

client = SpeckleClient(host="app.speckle.systems")

# Replace this with your actual personal access token (PAT), could also use default account method
personal_access_token = "e84c64a17e9103569a55ccb97e2f00526982240229"  

client.authenticate_with_token(personal_access_token)

stream_id = "2a3df00e3e" # Your specified stream ID
transport = ServerTransport(client=client, stream_id=stream_id)

object_id = send(base=speckle_columns, transports=[transport])
print(f"All columns with colors sent to Speckle with object ID: {object_id}")
print(f"View it at: https://app.speckle.systems/streams/{stream_id}/objects/{object_id}")

All columns with colors sent to Speckle with object ID: afe705cecf50a513a093a6c5935bf717
View it at: https://app.speckle.systems/streams/2a3df00e3e/objects/afe705cecf50a513a093a6c5935bf717


# 6. Example 3 : slabs
Finally, let's have a look of the other possibilities, for example with the slabs. We just have to adapt few scaling factors but the principle remains the same.


In [119]:
# Define a scaling factor to convert from millimeters to meters
scale_factor = 0.001  # Convert from mm to m

# Function to calculate bounding box dimensions with a scaling factor for slabs
def calculate_bounding_box(vertices):
    if not vertices or not isinstance(vertices, (tuple, list)):
        return None, None, None

    vertices_list = [(vertices[i], vertices[i + 1], vertices[i + 2]) for i in range(0, len(vertices), 3)]

    x_values = [v[0] for v in vertices_list]
    y_values = [v[1] for v in vertices_list]
    z_values = [v[2] for v in vertices_list]

    min_x, max_x = min(x_values), max(x_values)
    min_y, max_y = min(y_values), max(y_values)
    min_z, max_z = min(z_values), max(z_values)

    # Calculate the bounding box dimensions in meters
    length = (max_x - min_x) * scale_factor
    width = (max_y - min_y) * scale_factor
    height = (max_z - min_z) * scale_factor

    return length, width, height


# 2. Extract only slabs with PredefinedType FLOOR
floor_slabs = [slab for slab in ifc_file.by_type("IfcSlab") if slab.PredefinedType == "FLOOR"]
if len(floor_slabs) == 0:
    raise Exception("No floor slabs found in the IFC file.")

type_colors = {}

speckle_slabs = Base()

# Function to calculate the absolute placement in Z direction
def get_absolute_z_offset(placement):
    if placement is None:
        return 0
    # Start with the local Z offset
    z_offset = placement.RelativePlacement.Location.Coordinates[2]
    # If there's a PlacementRelTo, recursively add its offset
    if hasattr(placement, 'PlacementRelTo') and placement.PlacementRelTo:
        z_offset += get_absolute_z_offset(placement.PlacementRelTo)
    return z_offset

# 5. Loop over all floor slabs and extract geometry, applying color based on ObjectType
for slab in floor_slabs:
    settings = ifcopenshell.geom.settings()  
    shape = ifcopenshell.geom.create_shape(settings, slab)

    vertices = shape.geometry.verts
    faces = shape.geometry.faces

    if not vertices or not faces:
        print(f"No valid geometry found for slab: {slab.Name}")
        continue  

    dimensions = calculate_bounding_box(vertices)
    if dimensions is None:
        print(f"Could not calculate dimensions for slab: {slab.Name}")
        continue

    length, width, height = dimensions
    length = round(length, 2)
    width = round(width, 2)
    height = round(height, 2)

    # Get the slab's absolute placement elevation (Z-coordinate) using the recursive function
    placement = slab.ObjectPlacement
    z_offset = get_absolute_z_offset(placement) * scale_factor  # Convert Z offset to meters

    # Apply the Z offset to all vertices and convert to meters
    adjusted_vertices = []
    for i in range(0, len(vertices), 3):
        adjusted_vertices.extend([
            vertices[i] * scale_factor,               # X-coordinate in meters
            vertices[i + 1] * scale_factor,           # Y-coordinate in meters
            (vertices[i + 2] + z_offset) * scale_factor  # Z-coordinate in meters with offset
        ])

    object_type = getattr(slab, 'ObjectType', None)
    if not object_type:
        object_type = slab.Name if slab.Name else "Unnamed"

    key = (object_type)

    if key not in type_colors:
        type_colors[key] = generate_random_color() 

    formatted_faces = []
    for i in range(0, len(faces), 3):
        formatted_faces.extend([3, faces[i], faces[i + 1], faces[i + 2]])

    speckle_mesh = Mesh(
        name=f"IFC Floor Slab - {slab.Name}",
        vertices=adjusted_vertices,
        faces=formatted_faces
    )

    speckle_mesh.colors = [type_colors[key]] * (len(adjusted_vertices) // 3) 

    speckle_mesh['length'] = length*1000  # Add length as a direct attribute in meters
    speckle_mesh['width'] = width*1000  # Add width as a direct attribute in meters

    slab_object = Base()
    slab_object['mesh'] = speckle_mesh
    slab_object['guid'] = slab.GlobalId if slab.GlobalId else "No GUID"
    slab_object['name'] = slab.Name if slab.Name else "Unnamed Floor Slab"
    slab_object['ifc_type'] = slab.is_a()  # e.g., IfcSlab
    slab_object['length'] = length*1000  # Add length to the slab object in meters
    slab_object['width'] = width*1000  # Add width to the slab object in meters
    slab_object['object_type'] = object_type  # Add ObjectType to the slab object

    # Add this slab object to the main collection of slabs
    speckle_slabs[f'slab_{slab.GlobalId}'] = slab_object

client = SpeckleClient(host="app.speckle.systems")

# Replace this with your actual personal access token (PAT), could also use default account method
personal_access_token = "e84c64a17e9103569a55ccb97e2f00526982240229"  

client.authenticate_with_token(personal_access_token)

stream_id = "2a3df00e3e"  # Your specified stream ID
transport = ServerTransport(client=client, stream_id=stream_id)

object_id = send(base=speckle_slabs, transports=[transport])
print(f"All floor slabs with colors sent to Speckle with object ID: {object_id}")
print(f"View it at: https://app.speckle.systems/streams/{stream_id}/objects/{object_id}")

All floor slabs with colors sent to Speckle with object ID: d5deb8e4e5ec8320ed2da0c3ff559dc1
View it at: https://app.speckle.systems/streams/2a3df00e3e/objects/d5deb8e4e5ec8320ed2da0c3ff559dc1
